In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define el path una sola vez
path = '/content/drive/MyDrive/UNLP_Machine_Learning/'

# Ruta para la carpeta interna que contiene los archivos CSV
path_data = path + 'data/'

# Predicting wages

## upload data

In [ ]:
df = pd.read_csv(path_data+'df_clean.csv')
df

,directorio,secuencia_p,orden,clase,mes,estrato1,sex,age,p6050,p6090,...,y_viaticos_m,y_accidentes_m,y_salarySec_m,y_ingLab_m_ha,y_gananciaNeta_m,y_gananciaNetaAgro_m,y_gananciaIndep_m,y_gananciaIndep_m_hu,y_total_m,y_total_m_ha
0,4514331,1,2,1,1,2,0,29,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4514331,1,1,1,1,2,1,36,1,1,...,NaN,NaN,NaN,8404.320312,NaN,NaN,NaN,NaN,1.620833e+06,8404.320312
2,4514332,1,4,1,1,2,1,4,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4514332,1,3,1,1,2,1,7,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4514332,1,1,1,1,2,0,32,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32172,4804454,1,2,1,12,2,0,24,2,1,...,NaN,NaN,NaN,3345.555664,NaN,NaN,NaN,NaN,1.003667e+06,3345.555664
32173,4804455,1,1,1,12,3,0,36,1,1,...,500000.0,NaN,NaN,25958.333984,NaN,NaN,NaN,NaN,6.675000e+06,25958.333984
32174,4804455,1,2,1,12,3,1,41,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32175,4804455,1,3,1,12,3,1,8,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Split Sample

In [ ]:


# Verificar las primeras filas para confirmar la estructura
print(df.head())

# Identificar la variable objetivo y las características
target = 'y_total_m'  # Ajusta si es necesario
X = df.drop(columns=[target])
y = df[target]

# Separar las columnas numéricas y categóricas
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"Columnas numéricas: {numerical_cols}")
print(f"Columnas categóricas: {categorical_cols}")

# Preprocesamiento: Escalar numéricas y codificar categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=123
)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")


Explicación:

Carga de Datos: Cargamos el archivo CSV con los datos. Asegúrate de que el archivo se encuentra en el directorio correcto.

Identificación de Variables: Definimos y_total_m como la variable objetivo y el resto como características predictoras.

Separación de Columnas: Dividimos las columnas en numéricas y categóricas para aplicar diferentes técnicas de preprocesamiento.

Preprocesamiento: Utilizamos StandardScaler para escalar las variables numéricas y OneHotEncoder para codificar las variables categóricas.

División de Datos: Dividimos los datos en conjuntos de entrenamiento (70%) y prueba (30%) con random_state=123 para reproducibilidad.

3(b). Reportar y comparar el desempeño predictivo (RMSE) de al menos diez especificaciones
Implementaremos diez modelos de machine learning diferentes y calcularemos el RMSE para cada uno.


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import warnings

# Ignorar warnings para mayor claridad
warnings.filterwarnings('ignore')

# Definir los modelos a evaluar
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Decision Tree': DecisionTreeRegressor(random_state=123),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=123),

}

# Crear un DataFrame para almacenar los resultados
results = pd.DataFrame(columns=['Model', 'RMSE'])

# Evaluar cada modelo
for name, model in models.items():
    # Crear una pipeline que incluye el preprocesamiento y el modelo
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Entrenar el modelo
    pipeline.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = pipeline.predict(X_test)

    # Calcular el RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Almacenar los resultados
    results = results.append({'Model': name, 'RMSE': rmse}, ignore_index=True)

# Ordenar los resultados por RMSE ascendente
results = results.sort_values(by='RMSE').reset_index(drop=True)

print("Comparación de modelos por RMSE:\n")
print(results)


Explicación:

Definición de Modelos: Seleccionamos diez modelos diferentes que abarcan desde modelos lineales hasta ensambles y redes neuronales.

Pipeline: Para cada modelo, creamos una Pipeline que primero preprocesa los datos y luego aplica el modelo de regresión.

Entrenamiento y Predicción: Entrenamos cada modelo con los datos de entrenamiento y realizamos predicciones en el conjunto de prueba.

Cálculo de RMSE: Calculamos el Error Cuadrático Medio de la Raíz (RMSE) para evaluar el desempeño de cada modelo.

Almacenamiento y Ordenamiento: Almacenamos los resultados en un DataFrame y los ordenamos para facilitar la comparación.

3(c). Discusión de los resultados i. Sobre el desempeño general de los modelos Observa los RMSE obtenidos para cada modelo en el DataFrame results. Los modelos con menor RMSE tienen un mejor desempeño predictivo. En general, los modelos de ensamble como XGBoost, Gradient Boosting y Random Forest suelen tener un desempeño superior debido a su capacidad para capturar relaciones no lineales y manejar interacciones entre variables.

ii. Sobre la especificación con el menor error de predicción Supongamos que XGBoost es el modelo con el menor RMSE. Este modelo es conocido por su eficiencia y precisión en tareas de regresión y clasificación.

iii. Explorar observaciones que "fallaron el objetivo" Analizaremos los errores de predicción del mejor modelo para identificar posibles outliers o patrones que el modelo no ha capturado adecuadamente.

Código:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Seleccionar el mejor modelo
best_model_name = results.loc[0, 'Model']
best_model = models[best_model_name]

# Crear la pipeline para el mejor modelo
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', best_model)
])

# Entrenar el mejor modelo con todo el conjunto de entrenamiento
best_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_best = best_pipeline.predict(X_test)

# Calcular los errores de predicción
errors = y_test - y_pred_best

# Añadir los errores al DataFrame de prueba
test_results = X_test.copy()
test_results[target] = y_test
test_results['Predicted'] = y_pred_best
test_results['Error'] = errors

# Describir la distribución de errores
error_distribution = test_results['Error'].describe()
print("\nDistribución de errores de predicción:\n")
print(error_distribution)

# Visualizar la distribución de errores
plt.figure(figsize=(10,6))
sns.histplot(errors, kde=True, bins=30)
plt.title(f'Distribución de Errores de Predicción para {best_model_name}')
plt.xlabel('Error (Real - Predicho)')
plt.ylabel('Frecuencia')
plt.show()

# Identificar outliers (errores que están más allá de 3 desviaciones estándar)
threshold = 3 * errors.std()
outliers = test_results[np.abs(errors) > threshold]

print(f"\nNúmero de outliers (errores > 3σ): {outliers.shape[0]}")
print(outliers[['Predicted', target, 'Error']].head())
